In [14]:
import os

In [15]:
#docu1.py
#PURPOSE: To save csv generated data from docusign to aws database.

In [16]:
#CHECK IF FILE "Envelope Recipient Report.csv" EXIST
def file_readiness_checker():
    #File "Envelope Recipient Report.csv" extracted from Docusign
    print("Checking if 'Envelope Recipient Report.csv' exist")
    if os.path.exists("Envelope Recipient Report.csv"):
        print("'Envelope Recipient Report.csv' exist.")
        print("Checking if update is available.")
        #Function to Check if update is available
        
    else:
        print("'Envelope Recipient Report.csv' does not exist.")



In [17]:
file_checker()

Checking if 'Envelope Recipient Report.csv' exist
'Envelope Recipient Report.csv' exist.
Proceeding to update database.
